In [1]:
# %load command.py

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

%config InlineBackend.figure_format='svg'
plt.rcParams['figure.dpi']=120

pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_colwidth', None)


![Comparison of json and dict](jsonDict.png)

https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd

1. Flattening a simple JSON <br>
2. Flattening a JSON with multiple levels<br>
3. Flattening a JSON with a nested list<br>
4. Ignoring KeyError if keys are not always present<br>
5. Custom separator using sep<br>
6. Adding prefix for meta and record data<br>
7. Working with a local file<br>
8. Working with a URL

## 1. Flattening a simple JSON

In [2]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
}

df=pd.json_normalize(a_dict)
df

,school,location,ranking
0,ABC primary school,London,2


In [3]:
json_list = [
    { 'class': 'Year 1', 'student number': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'student number': 25, 'room': 'Blue' },
]

df=pd.json_normalize(json_list)

df

,class,student number,room
0,Year 1,20,Yellow
1,Year 2,25,Blue


In [4]:
json_list = [
    { 'class': 'Year 1', 'num_of_students': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'room': 'Blue' }, # no num_of_students
]
pd.json_normalize(json_list)

,class,num_of_students,room
0,Year 1,20.00,Yellow
1,Year 2,NaN,Blue


## 2. Flattening a JSON with Multiple levels

In [5]:
# The value of info is multiple levels (known as a nested dict).
json_obj = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    }
}

pd.json_normalize(json_obj)

,school,location,ranking,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,John Kasich,admission@abc.com,info@abc.com,123456789


**If you don’t want to dig all the way down to each value use the max_level argument. With the argument max_level=1, we can see that our nested value contacts is put up into a single column info.contacts.**

In [6]:
pd.json_normalize(json_obj, max_level=1)

,school,location,ranking,info.president,info.contacts
0,ABC primary school,London,2,John Kasich,"{'email': {'admission': 'admission@abc.com', 'general': 'info@abc.com'}, 'tel': '123456789'}"


In [7]:
# When the data is a list of dicts
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        }
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        }
    },
]
pd.json_normalize(json_list)

,class,student count,room,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,Rick Scott,Elon Mask
1,Year 2,25,Blue,Alan Turing,Albert Einstein


In [8]:
pd.json_normalize(json_list, max_level=1)

,class,student count,room,info.teachers
0,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Elon Mask'}"
1,Year 2,25,Blue,"{'math': 'Alan Turing', 'physics': 'Albert Einstein'}"


## 3. Flattening JSON with a nested list

#### When the data is a dict

In [9]:
json_obj = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    },
    'students': [
      { 'name': 'Tom' },
      { 'name': 'James' },
      { 'name': 'Jacqueline' }
    ],
}

pd.json_normalize(json_obj)

,school,location,ranking,students,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,"[{'name': 'Tom'}, {'name': 'James'}, {'name': 'Jacqueline'}]",John Kasich,admission@abc.com,info@abc.com,123456789


In [11]:
# Flatten students
pd.json_normalize(json_obj, record_path =['students'])

,name
0,Tom
1,James
2,Jacqueline


In [13]:
# Include meta data
pd.json_normalize(
    json_obj, 
    record_path =['students'],
    meta=['school', ['info', 'contacts', 'tel']],
)

,name,school,info.contacts.tel
0,Tom,ABC primary school,123456789
1,James,ABC primary school,123456789
2,Jacqueline,ABC primary school,123456789


### A list of dicts

In [14]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        },
        'students': [
            { 
                'name': 'Tom', 
                'sex': 'M', 
                'grades': { 'math': 66, 'physics': 77 } 
            },
            { 
                'name': 'James', 
                'sex': 'M', 
                'grades': { 'math': 80, 'physics': 78 } 
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(json_list)

,class,student count,room,students,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,"[{'name': 'Tom', 'sex': 'M', 'grades': {'math': 66, 'physics': 77}}, {'name': 'James', 'sex': 'M', 'grades': {'math': 80, 'physics': 78}}]",Rick Scott,Elon Mask
1,Year 2,25,Blue,"[{'name': 'Tony', 'sex': 'M'}, {'name': 'Jacqueline', 'sex': 'F'}]",Alan Turing,Albert Einstein


In [15]:
pd.json_normalize(json_list, record_path =['students'])

,name,sex,grades.math,grades.physics
0,Tom,M,66.00,77.00
1,James,M,80.00,78.00
2,Tony,M,NaN,NaN
3,Jacqueline,F,NaN,NaN


In [16]:
pd.json_normalize(
    json_list, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']]
)

,name,sex,grades.math,grades.physics,class,room,info.teachers.math
0,Tom,M,66.00,77.00,Year 1,Yellow,Rick Scott
1,James,M,80.00,78.00,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


## 4. Ignoring KeyError if keys are not always present

In [17]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask',
            }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M' },
            { 'name': 'James', 'sex': 'M' },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'physics': 'Albert Einstein'
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    errors='ignore'
)

,name,sex,class,room,info.teachers.math
0,Tom,M,Year 1,Yellow,Rick Scott
1,James,M,Year 1,Yellow,Rick Scott
2,Tony,M,Year 2,Blue,NaN
3,Jacqueline,F,Year 2,Blue,NaN


## 5. Custom separator using `sep`

In [18]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 'math': 'Rick Scott', 'physics': 'Elon Mask' }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M', 'grades': { 'math': 66, 'physics': 77 } },
            { 'name': 'James', 'sex': 'M', 'grades': { 'math': 80, 'physics': 78 } },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 'math': 'Alan Turing', 'physics': 'Albert Einstein' }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    sep='->'
)

,name,sex,grades->math,grades->physics,class,room,info->teachers->math
0,Tom,M,66.00,77.00,Year 1,Yellow,Rick Scott
1,James,M,80.00,78.00,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


**Adding prefix for meta and record data using meta_prefix and record_prefix**

In [ ]:


pd.json_normalize(
    data, 
    record_path=['students'], 
    meta=['class'],
    meta_prefix='meta-',
    record_prefix='student-'
)

## 6. Working with a local file

In [21]:
import json
# load data using Python JSON module
with open('./pandasData/simple.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78
